In [4]:
import modern_robotics as mr
import numpy as np
import sympy as sp
from sympy import*
from sympy.physics.mechanics import dynamicsymbols, mechanics_printing
mechanics_printing()

### Utilities

In [127]:
def exp3(omega, theta):
    omega = skew(omega)
    R = sp.eye(3) + sp.sin(theta) * omega + (1 - sp.cos(theta)) * omega * omega
    return R

def skew(v):
    return Matrix([[0, -v[2], v[1]],
                    [v[2], 0, -v[0]],
                    [-v[1], v[0], 0]])
                    
def exp6(twist, theta):
    omega = skew(twist[:3])
    v = Matrix(twist[3:])
    T = eye(4)
    T[:3,:3] = exp3(twist[:3], theta)
    T[:3,3] = (eye(3) * theta + (1 - cos(theta)) * omega +
              (theta-sin(theta)) * omega * omega) * v
    return T

def Ad(T):
    AdT = sp.zeros(6)
    R = sp.Matrix(T[:3, :3])
    AdT[:3, :3] = R
    AdT[3:, 3:] = R
    AdT[3:, :3] = skew(T[:3, 3]) * R
    return AdT

def rotX(theta):
    Rx = sp.eye(4)
    Rx[1,1] =    sp.cos(theta)
    Rx[1,2] =   -sp.sin(theta)
    Rx[2,1] =    sp.sin(theta)
    Rx[2,2] =    sp.cos(theta)
    return Rx

def rotZ(theta):
    Rz = sp.eye(4)
    Rz[0,0] =    sp.cos(theta)
    Rz[0,1] =   -sp.sin(theta)
    Rz[1,0] =    sp.sin(theta)
    Rz[1,1] =    sp.cos(theta)
    return Rz

def PsFromTsd(Tsd):
    #Finner Ps fra T_sd
    #T_sd gir konfigurasjonen vi vil ha end-effector framen, B, i.
    #B, og derav også M, er lik som i DH
    #s er plassert nederst på roboten med positiv z oppover, altså ikke som i DH. Bør kanskje endres til å være lik DH 
    Pd = np.array([0,0,80,1])
    Ps = Tsd@Pd

    return Ps


### Task 3-3

In [167]:
#Definerer S og M
"""
S1 = np.array([0,0,-1,0,0,0])
S2 = np.array([0,1,0,-400,0,25])
S3 = np.array([0,1,0,-855,0,25])
S4 = np.array([-1,0,0,0,-890,0])
S5 = np.array([0,1,0,-890,0,445])
S6 = np.array([-1,0,0,0,-890,0])
Slist = np.array([S1,S2,S3,S4,S5,S6]).T
print(Slist)

M = np.array([[0,0,-1,525],
              [0,1,0,0],
              [1,0,0,890],
              [0,0,0,1]])

thetasUp = [0,0,0,0,0,0]
thetasDown = [0,0,0,0,0,0]


#Limits til roboten slik den er gitt i oppgaven. Antar at ledd 5 har limit på +-90
theta_limits = [[-180,180],[-190+90,45+90],[-120-90, 156-90],[-180,180],[-90,90],[-180,180]]
"""

S1 = np.array([0,0,1,0,0,0])
S2 = np.array([0,-1,0,0,0,-25])
S3 = np.array([0,-1,0,-455,0,-25])
S4 = np.array([-1,0,0,0,490,0])
S5 = np.array([0,-1,0,-490,0,-445])
S6 = np.array([-1,0,0,0,490,0])
Slist = np.array([S1,S2,S3,S4,S5,S6]).T
print(Slist)

M = np.array([[0,0,-1,525],
              [0,-1,0,0],
              [-1,0,0,-490],
              [0,0,0,1]])

thetasUp = [0,0,0,0,0,0]
thetasDown = [0,0,0,0,0,0]


[[   0    0    0   -1    0   -1]
 [   0   -1   -1    0   -1    0]
 [   1    0    0    0    0    0]
 [   0    0 -455    0 -490    0]
 [   0    0    0  490    0  490]
 [   0  -25  -25    0 -445    0]]


In [224]:
#Her endres thetasGen for å teste forskjellige konfigurasjoner:
thetasGen = np.array([3,3,1,2,1,0])
Tsd = mr.FKinSpace(M,Slist,thetasGen)
print("T_sd\n", Tsd)


T_sd
 [[ 4.4539e-01 -7.4000e-01 -5.0401e-01  2.5001e+02]
 [ 4.3277e-01 -3.1487e-01  8.4473e-01 -9.7469e+01]
 [-7.8380e-01 -5.9436e-01  1.8001e-01  1.4107e+02]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  1.0000e+00]]


In [225]:

Ps = PsFromTsd(Tsd)
print("Ps", Ps)
Psmerket = [Ps[0], Ps[1], Ps[2]]

#theta1
thetasUp[0] = -atan2(-Psmerket[1],Psmerket[0]) #minus foran fordi vinkelen er definert andre vei ##
thetasDown[0] = thetasUp[0]

#theta2 and theta3
a = np.sqrt(420**2+35**2)
c = 455
b = np.sqrt((np.sqrt(Psmerket[0]**2+Psmerket[1]**2)-25)**2 + Psmerket[2]**2)
print("a",a,"c",c,"b", b)
print("d", np.sqrt((np.sqrt(Psmerket[0]**2+Psmerket[1]**2)-25)**2))

psi = np.arccos(420/a) #Vinkelen mellom den faktiske armen og den vi tegna for å få en trekant(Pga 35mm offset i elbow). Se notatbok
phi = atan2(-Psmerket[2], sqrt(Psmerket[0]**2 + Psmerket[1]**2)-25) ##
print("args", -Psmerket[2], sqrt(Psmerket[0]**2 + Psmerket[1]**2)-25)

alpha = np.arccos((b**2+c**2-a**2)/(2*b*c))
beta = np.arccos((a**2+c**2-b**2)/(2*a*c))

print("alpha:", np.rad2deg(alpha), "beta:", np.rad2deg(beta), "phi:", phi, "psi:", np.rad2deg(psi))
thetasUp[1] = np.pi/2 - (alpha + phi)
thetasDown[1] = np.pi/2 - (phi-alpha)

print(thetasUp[1])

thetasUp[2] = np.pi/2 - (beta-psi)
thetasDown[2] = -(2*np.pi - (beta+psi) - np.pi/2)

print(thetasUp, thetasDown)

#Vi har XYX euler angles. De er egentlig (-X)Y(-X) fordi det er slik S'ene er definert,.

#Elbow down:
T1 = exp6(S1, -thetasDown[0])
T2 = exp6(S2, -thetasDown[1])
T3 = exp6(S3, -thetasDown[2])

R = (T3@T2@T1@Tsd@np.linalg.inv(M)) #R er den resterende rotasjonen vi ønsker å få fra de tre siste leddene, definert i s

thetasDown[3] = -atan2(R[1,0], -R[2,0]) #minus foran theta4 og 6 fordi de er i minus x retning
thetasDown[4] = -atan2(sqrt(1-R[0,0]**2), R[0,0])
thetasDown[5] = -atan2(R[0,1], R[0,2])


#Elbow up:
T1 = exp6(S1, -thetasUp[0])
T2 = exp6(S2, -thetasUp[1])
T3 = exp6(S3, -thetasUp[2])

R = (T3@T2@T1@Tsd@np.linalg.inv(M))

thetasUp[3] = -atan2(R[1,0], -R[2,0])
thetasUp[4] = -atan2(sqrt(1-R[0,0]**2), R[0,0])
thetasUp[5] = -atan2(R[0,1], R[0,2])

Ps [209.6891 -29.8905 155.4671   1.    ]
a 421.4558102577303 c 455 b 243.03814022740696
d 186.8088044312549
args -155.46706465409926 186.808804431255
alpha: 66.42135383932616 beta: 31.905434198839078 phi: -0.694084160821108 psi: 4.763641690726143
1.10560916948759
[-0.141592653589793, 1.10560916948759, 1.097082682635196, 1.09683919743480, 1.03515054185725 - pi, -0.0851244511188330] [-0.141592653589793, 3.42415180574442, -4.072392872448108, -1.2790915209693 + pi, -0.925458806156948, 0.239742346967086 - pi]


In [226]:
#testing av analytisk løsning:

#UP
thetasUpN = np.zeros(6)
thetasUpDeg = np.zeros(6)
for i in range(0,6):
    thetasUpN[i] = N(thetasUp[i])
    thetasUpDeg[i] = np.rad2deg(thetasUpN[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

TupThetas = mr.FKinSpace(M,Slist,thetasUpN)
PUpReached = PsFromTsd(TupThetas)
#print(P_reached, P_s)

#DOWN
thetasDownN = np.zeros(6)
thetasDownDeg = np.zeros(6)
for i in range(0,6):
    thetasDownN[i] = N(thetasDown[i])
    thetasDownDeg[i] = np.rad2deg(thetasDownN[i])

#print(thetas_deg, np.rad2deg(thetas_gen))

TDownThetas = mr.FKinSpace(M,Slist,thetasDownN)
PDownReached = PsFromTsd(TDownThetas)


#fk_test = exp6(S4,thetas_num[3])@exp6(S5,thetas_num[4])@exp6(S6,thetas_num[5])@M
#R_test = rotX(thetas_num[3])@rotY(thetas_num[4])@rotX(thetas_num[5]) 
thetasCalc, asd = mr.IKinSpace(Slist,M,Tsd,[0,0,0,0,0,0],0.01,0.01)

Tsd, thetasGen, TupThetas, thetasUpN, TDownThetas, thetasDownN, Ps, PUpReached, PDownReached

(array([[ 4.4539e-01, -7.4000e-01, -5.0401e-01,  2.5001e+02],
        [ 4.3277e-01, -3.1487e-01,  8.4473e-01, -9.7469e+01],
        [-7.8380e-01, -5.9436e-01,  1.8001e-01,  1.4107e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([3, 3, 1, 2, 1, 0]),
 array([[ 4.4539e-01, -7.4000e-01, -5.0401e-01,  2.5001e+02],
        [ 4.3277e-01, -3.1487e-01,  8.4473e-01, -9.7469e+01],
        [-7.8380e-01, -5.9436e-01,  1.8001e-01,  1.4107e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([-0.1416,  1.1056,  1.0971,  1.0968, -2.1064, -0.0851]),
 array([[ 4.4539e-01, -7.4000e-01, -5.0401e-01,  2.5001e+02],
        [ 4.3277e-01, -3.1487e-01,  8.4473e-01, -9.7469e+01],
        [-7.8380e-01, -5.9436e-01,  1.8001e-01,  1.4107e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([-0.1416,  3.4242, -4.0724,  1.8625, -0.9255, -2.9019]),
 array([209.6891, -29.8905, 155.4671,   1.    ]),
 array([209.6891, -29.8905, 155.4671,   1.  

In [227]:
Tsd, thetasUpN, thetasDownN


(array([[ 4.4539e-01, -7.4000e-01, -5.0401e-01,  2.5001e+02],
        [ 4.3277e-01, -3.1487e-01,  8.4473e-01, -9.7469e+01],
        [-7.8380e-01, -5.9436e-01,  1.8001e-01,  1.4107e+02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]]),
 array([-0.1416,  1.1056,  1.0971,  1.0968, -2.1064, -0.0851]),
 array([-0.1416,  3.4242, -4.0724,  1.8625, -0.9255, -2.9019]))